# Pulizia dati del file "Movies.csv"

In [1]:
import pandas as pd
import os

In [17]:
import os
import pandas as pd

# Ottiene la directory di lavoro corrente
script_dir = os.getcwd()

# Risali alla cartella "Solutions"
solutions_dir = os.path.abspath(os.path.join(script_dir, ".."))

# Imposta "Data" come directory di lavoro
data_dir = os.path.join(solutions_dir, "Data")
os.chdir(data_dir)  # Cambia la directory di lavoro

# Ora aggiorna la directory corrente dopo il cambio
script_dir = os.getcwd()

# Costruisce il percorso relativo al file CSV
file_path = os.path.join(script_dir, "main_data", "movies.csv")

# Costruisce il percorso relativo al file CSV pulito
clean_path = os.path.join(script_dir, "clean_data")

# Crea la cartella clean_data se non esiste
if not os.path.exists(clean_path):
    os.makedirs(clean_path)

# Controlla che il file esista prima di caricarlo
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Il file non esiste: {file_path}")

# Carica il CSV in un DataFrame
df = pd.read_csv(file_path)
df.head()

,id,name,date,tagline,description,minute,rating
0,1000001,Barbie,2023.0,She's everything. He's just Ken.,Barbie and Ken are having the time of their li...,114.0,3.86
1,1000002,Parasite,2019.0,Act like you own the place.,"All unemployed, Ki-taek's family takes peculia...",133.0,4.56
2,1000003,Everything Everywhere All at Once,2022.0,The universe is so much bigger than you realize.,An aging Chinese immigrant is swept up in an i...,140.0,4.30
3,1000004,Fight Club,1999.0,Mischief. Mayhem. Soap.,A ticking-time-bomb insomniac and a slippery s...,139.0,4.27
4,1000005,La La Land,2016.0,Here's to the fools who dream.,"Mia, an aspiring actress, serves lattes to mov...",129.0,4.09


In [18]:
# Rimuove duplicati
df.drop_duplicates(inplace=True)

# Rimuove righe con troppi valori NaN
df.dropna(thresh=df.shape[1] - 2, inplace=True)

# Converti "data" e "anno" in numerico (float se ci sono NaN)
df["minute"] = pd.to_numeric(df["minute"], errors="coerce")
df["date"] = pd.to_numeric(df["date"], errors="coerce")

# Dopo la pulizia, rimuovi eventuali NaN e riconverti in int
df.dropna(subset=["minute", "date"], inplace=True)  # Elimina righe con NaN in questi campi
df["minute"] = df["minute"].astype(int)
df["date"] = df["date"].astype(int)


In [19]:
# Salva il file pulito
cleaned_csv_path = os.path.join(clean_path, "movies_cleaned.csv")
df.to_csv(cleaned_csv_path, index=False, encoding="utf-8")

# Caricamento dati nel DB

In [5]:
import psycopg2
from sqlalchemy import create_engine, text

In [6]:
#  Parametri di connessione PostgreSQL
DB_NAME = "ium_database"  
DB_USER = "postgres"       
DB_PASSWORD = "password"   
DB_HOST = "localhost"      
DB_PORT = "5432"    

conn = psycopg2.connect(dbname="postgres", user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT) # Crea una connessione al database usando psycopg2.connect
conn.autocommit = True # Conferma automaticamente ogni operazione evitando la conferma manuale dopo ogni query
cur = conn.cursor() # Crea un oggetto cur che permette di eseguire query SQL sul database
cur.execute(f"CREATE DATABASE {DB_NAME};") # Esegue il comando per creare il database con il nome specificato

DuplicateDatabase: ERRORE:  il database "ium_database" esiste già


In [7]:
# Crea un oggetto di connessione a un database PostgreSQL utilizzando SQLAlchemy
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [8]:
# Query SQL per creare la tabella
create_table_query = text("""
    CREATE TABLE IF NOT EXISTS actors_data (
        id INT,
        name TEXT,
        role TEXT
    );
""")

# Eseguire la query
with engine.connect() as connection: # Apre una connessione al database usando SQLAlchemy
    connection.execute(create_table_query) # Esegue il comando per la creazione della 
    connection.commit() # Salva le modifiche

In [9]:
# Legge il CSV pulito
df = pd.read_csv(cleaned_csv_path)

# Carica i dati nella tabella PostgreSQL
df.to_sql("movies_data", engine, if_exists="replace", index=False) # Esporta il dataframe nel database, sovrascrive e aggiunge i nuovi dati alla tabella esistente

924